# ETL I Pair 23/08/23

In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" 

import requests
import pandas as pd
import numpy as np

1. Nuestros jefes están especialmente interesados en los datos de generación de las energías renovables y no renovables a nivel nacional. En concreto, nos piden sacar toda la información para las fechas de 1 de Enero del 2011 hasta el 31 de Diciembre del 2022. De toda la información que tenemos de generación queremos la evolucion de renovable y no renovable. Además deberemos incluir una columna nueva identificando si el dato corresponde a energía renovable o no renovable. NOTA: nos piden que el código debe estar en una función. Al final deberemos tener un dataframe como el siguiente:

In [2]:
url_api = "https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date=2011-01-01T00:00&end_date=2011-12-31T23:59&time_trunc=day"
response = requests.get(url=url_api)
response.status_code

200

In [3]:
# response.reason
response.json().keys()
type(response.json()['included'])
len(response.json()['included']) # el dicc "included" tiene otros 2 diccionarios, uno para el type 'Renovable' y otro para el type 'No Renovable'
response.json()['included'][0].keys()
response.json()['included'][0]['type']
response.json()['included'][0]['attributes'].keys()
type(response.json()['included'][0]['attributes']['values'])
response.json()['included'][0]['attributes']['values'][0].keys()

response_data = response.json()['included']
renovable = response_data[0]
df = pd.json_normalize(renovable['attributes']['values'])
# df["tipo_energia"] = renovable['attributes']['type']
df

dict_keys(['data', 'included'])

list

2

dict_keys(['type', 'id', 'groupId', 'attributes'])

'Renovable'

dict_keys(['title', 'description', 'color', 'type', 'magnitude', 'composite', 'last-update', 'values'])

list

dict_keys(['value', 'percentage', 'datetime'])

,value,percentage,datetime
0,182024.9820,0.290121,2011-01-01T00:00:00.000+01:00
1,265705.8895,0.387418,2011-01-02T00:00:00.000+01:00
2,249489.5735,0.295983,2011-01-03T00:00:00.000+01:00
3,270981.7130,0.310937,2011-01-04T00:00:00.000+01:00
4,379833.0150,0.452824,2011-01-05T00:00:00.000+01:00
...,...,...,...
360,154055.4800,0.199828,2011-12-27T00:00:00.000+01:00
361,199755.3925,0.258876,2011-12-28T00:00:00.000+01:00
362,306070.1190,0.387523,2011-12-29T00:00:00.000+01:00
363,268552.8560,0.343830,2011-12-30T00:00:00.000+01:00


In [4]:
def sacar_info_energia():
    df_final = pd.DataFrame() # DataFrame vacío dónde vamos a almacenar la información
    for year in range(2011, 2023): # Loop para recoger desde 2011 - 2022
        url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=day" # URL formateada para cada año
        response = requests.get(url=url) # Petición de la información
        status_code = response.status_code # Respuesta de la petición
                
        if status_code == 200: # Si la petición devuelve un 200, vamos generando la información
            response_data = response.json()['included'] # La información se encuentra dentro de la key 'included'
            len_response_data = len(response_data) # Queremos saber cuántos elementos tiene la lista de dicc dentro de 'included'
            
            for i in range(len_response_data): # Recorremos cada elemento que hay en la lista 'response_data'. En este caso, hay 2 elementos.
                attributes_data = response_data[i]['attributes'] # Obtenemos el diccionario de la key 'attribute' para cada elemento de la lista (i)
                type_energy = attributes_data['type'] # Obtenemos el valor de la key 'type' (Renovable | No renovable)
                values_data = attributes_data['values'] # Obtenemos los valores de la key 'values' ('value', 'percentage' y 'datetime')
                df = pd.json_normalize(values_data) # Generamos el DataFrame con los valores normalizados de la key 'values'
                df["tipo_energia"] = type_energy # Añadimos la columna 'tipo_energia' con el valor de la key 'type'
                df_final = pd.concat([df_final, df], axis = 0) # Añadimos la información generada al DataFrame final que vamos a devolver
        else: # Si no devuelve un 200, no queremos hacer nada
            continue
    return df_final.reset_index(drop = True) # Devolvemos el DataFrame con la información obtenida

df = sacar_info_energia()

2. Sacaremos la misma información que en el ejericio anterior, pero en este caso por CCAA. Para esto necesitaremos los códigos de cada una de las comunidades autónomas que encontraréis en el siguiente diccionario:

In [5]:
cod_comunidades = {
    'Ceuta': 8744,
    'Melilla': 8745,
    'Andalucía': 4,
    'Aragón': 5,
    'Cantabria': 6,
    'Castilla - La Mancha': 7,
    'Castilla y León': 8,
    'Cataluña': 9,
    'País Vasco': 10,
    'Principado de Asturias': 11,
    'Comunidad de Madrid': 13,
    'Comunidad Foral de Navarra': 14,
    'Comunitat Valenciana': 15,
    'Extremadura': 16,
    'Galicia': 17,
    'Illes Balears': 8743,
    'Canarias': 8742,
    'Región de Murcia': 21,
    'La Rioja': 20
}

In [6]:
# Prueba For Loop Comunidades
for k,v in cod_comunidades.items():
    print(k,v)

Ceuta 8744
Melilla 8745
Andalucía 4
Aragón 5
Cantabria 6
Castilla - La Mancha 7
Castilla y León 8
Cataluña 9
País Vasco 10
Principado de Asturias 11
Comunidad de Madrid 13
Comunidad Foral de Navarra 14
Comunitat Valenciana 15
Extremadura 16
Galicia 17
Illes Balears 8743
Canarias 8742
Región de Murcia 21
La Rioja 20


In [7]:
def sacar_info_energia_ccaa(ccaa):
    df_final = pd.DataFrame() # DataFrame vacío dónde vamos a almacenar la información
    for year in range(2011, 2023): # Loop para recoger desde 2011 - 2022
        for k_nombre_ccaa, v_id_ccaa in ccaa.items():
            url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=day&geo_limit=ccaa&geo_ids={v_id_ccaa}" # URL formateada para cada año
            response = requests.get(url=url) # Petición de la información
            status_code = response.status_code # Respuesta de la petición
            
            if status_code == 200: # Si la petición devuelve un 200, vamos generando la información
                response_data = response.json()['included'] # La información se encuentra dentro de la key 'included'
                len_response_data = len(response_data) # Queremos saber cuántos elementos tiene la lista de dicc dentro de 'included'
                
                for i in range(len_response_data): # Recorremos cada elemento que hay en la lista 'response_data'. En este caso, hay 2 elementos.
                    attributes_data = response_data[i]['attributes'] # Obtenemos el diccionario de la key 'attribute' para cada elemento de la lista (i)
                    type_energy = attributes_data['type'] # Obtenemos el valor de la key 'type' (Renovable | No renovable)
                    values_data = attributes_data['values'] # Obtenemos los valores de la key 'values' ('value', 'percentage' y 'datetime')
                    df = pd.json_normalize(values_data) # Generamos el DataFrame con los valores normalizados de la key 'values'
                    df["tipo_energia"] = type_energy # Añadimos la columna 'tipo_energia' con el valor de la key 'type'
                    df["comunidad"] = k_nombre_ccaa # Añadimos el nombre de la Comunidad Autónoma
                    df["id_comunidad"] = v_id_ccaa # Añadimos el ID de la Comunidad Autónoma
                    df_final = pd.concat([df_final, df], axis = 0) # Añadimos la información generada al DataFrame final que vamos a devolver
            else: # Si no devuelve un 200, no queremos hacer nada
                continue
    return df_final.reset_index(drop = True) # Devolvemos el DataFrame con la información obtenida

df_ccaa = sacar_info_energia_ccaa(cod_comunidades)

- Guardad los csv ya que mañana los seguiremos usando en los siguientes pairs.

In [8]:
df.to_csv("data/pairETL1_datos_energia_nacional.csv")
df_ccaa.to_csv("data/pairETL1_datos_energia_ccaa.csv")